
### 실습 과제 1️⃣ : 타이타닉 생존자 분석 시각화 🚢

>타이타닉호 침몰 사건의 승객 데이터를 사용하여 생존에 영향을 미친 요인들을 시각적으로 분석합니다.
>  * **데이터**: `px.data.titanic()`를 사용하거나, 캐글 등에서 더 상세한 타이타닉 데이터셋(CSV)을 다운로드하여 사용할 수 있습니다. 여기서는 Plotly 내장 데이터를 사용합니다.
>  * **분석 목표 예시**:
>    1.  전체 생존자와 사망자 비율은 어떻게 되는가? (파이 차트)
>    2.  성별에 따른 생존율 차이는 있는가? (막대 그래프)
>    3.  객실 등급(Pclass)에 따른 생존율 차이는 있는가? (막대 그래프 또는 바이올린 플롯)
>    4.  나이 분포에 따른 생존자와 사망자 분포는 어떻게 다른가? (히스토그램 또는 박스플롯)
>    5.  탑승 항구(Embarked)별 생존율은? (막대 그래프)
>    6.  요금(Fare)과 생존 여부의 관계는? (박스플롯 또는 바이올린 플롯)

라이브러리 로드

In [5]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

데이터셋 로드

In [7]:
import seaborn as sns
titanic_df = sns.load_dataset('titanic')

In [8]:
# 타이타닉 데이터 로드
# titanic_df = px.data.titanic()
titanic_df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


데이터셋 전처리

In [4]:
# 결측치 처리 (Age, Embarked 등) - 여기서는 간단히 Age 평균으로 채우고 Embarked 최빈값으로 채움
# survived 컬럼을 문자열로 변경하여 범주형으로 취급 (0: 사망, 1: 생존)

titanic_df['age'] = titanic_df['age'].fillna(titanic_df['age'].mean())
titanic_df['embarked'] = titanic_df['embarked'].fillna(titanic_df['embarked'].mode()[0])

titanic_df['survived'] = titanic_df['survived'].map({0: '사망', 1: '생존'})
titanic_df.head()


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,사망,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,생존,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,생존,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,생존,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,사망,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


데이터셋 시각화1

In [11]:
# 1. 전체 생존자와 사망자 비율 (파이 차트)

# 생존자, 사망자 수
survival_counts = titanic_df['survived'].value_counts().reset_index()
survival_counts.columns = ['survived', 'count']

fig = px.pie(survival_counts,
             names= 'survived',
             values= 'count',
             title = '타이타닉 생존자/사망자 비율'
)

fig.show()

In [7]:
gender_survival_cnt = titanic_df.groupby(['sex', 'survived']).size().reset_index(name= 'count')
gender_survival_cnt.head()

,sex,survived,count
0,female,사망,81
1,female,생존,233
2,male,사망,468
3,male,생존,109


In [13]:
# 2. 성별에 따른 생존율 (막대 그래프)
# 먼저 성별, 생존여부로 그룹화하여 카운트
fig = px.bar(
    gender_survival_cnt,
    x= 'sex',
    y= 'count',
    color= 'survived',
    title= '성별에 따른 생존자',
    labels= {'sex': '성별', 'count': '생존자', 'survived': '생존여부'}
)

fig.show()


In [9]:
# 3. 객실 등급별 생존율 (바이올린 플롯 - 생존/사망자 수를 함께 보기 위해)

# 히스토그램, 바이올린 - 집계 x / 막대 - 집계 o
# 바이올린, 박스 플롯은 연속형 데이터 선택

fig = px.violin(
    titanic_df,
    x='class',
    y='age',
    color='survived',
    box=True,
    points='all',
    title='객실 등급별 요금 분포 및 생존 여부'
)

fig.show()


In [22]:
# 4. 나이 분포에 따른 생존자와 사망자 (히스토그램)

px.histogram(
    titanic_df,
    x= 'age',
    color= 'survived',
    nbins= 30,
    title= '나이 분포에 따른 생존/사망자',
)


In [29]:
# 5. 탑승 항구별 생존율 (막대 그래프, 비율로 표시)
port_survival = titanic_df.groupby(['embarked', 'survived']).size().unstack()
port_survival_pct = port_survival.div(port_survival.sum(axis=1), axis=0) * 100

fig = px.bar(
    port_survival_pct,
    title='탑승 항구별 생존율 (%)',
    labels= {'embarked': '탑승 항구', 'value': '비율 (%)', 'survived': '생존여부'}
)

fig.show()

In [31]:
# 6. 요금(Fare)과 생존 여부의 관계 (박스 플롯)
# Fare가 매우 큰 아웃라이어가 있을 수 있으므로, 상위 1% 제외 (예시)
fare_threshold = titanic_df['fare'].quantile(0.99)

# 임계값 이하의 데이터만 선택
fare_df = titanic_df[titanic_df['fare'] <= fare_threshold]

fig = px.box(
    fare_df,
    x= 'survived',
    y= 'fare',
    color= 'survived',
    title= '생존 여부에 따른 요금 분포 (상위 1% 제외)'
)

fig.show()


### 실습 과제 2️⃣ : Gapminder 국가별 변화 시각화 🌍

> `px.data.gapminder()` 데이터셋을 사용하여 여러 국가의 시간 경과에 따른 경제 및 사회 지표 변화를 탐색합니다.
>  * **분석 목표 예시**:
>    1.  시간에 따른 전 세계 국가들의 1인당 GDP와 기대수명의 변화를 애니메이션으로 시각화합니다. (애니메이션 산점도)
>    2.  특정 대륙(예: 아시아) 국가들의 인구 변화 추세를 비교합니다. (선 그래프)
>    3.  2007년 기준, 대륙별 평균 기대수명과 평균 1인당 GDP를 비교합니다. (막대 그래프 또는 산점도)
>    4.  연도별로 인구가 가장 많은 상위 5개 국가의 변화를 시각화합니다. (애니메이션 막대 경주 차트 - 고급)

라이브러리 로드
> 생략

데이터셋 로드

In [14]:
# Gapminder 데이터 로드
gapminder_df = px.data.gapminder()

데이터 시각화

In [16]:
# 1. 시간에 따른 GDP와 기대수명 변화 (애니메이션 산점도 - 앞서 실습)

fig = px.scatter(gapminder_df,
                 x= 'gdpPercap',
                 y= 'lifeExp',
                 animation_frame= 'year',
                 size= 'pop',
                 color= 'continent',
                 hover_name= 'country',
                 log_x=True,
                 range_y= [20, 90],
                 size_max= 60,
                 title= '연도별 GDP, 기대수명')

fig.show()

In [19]:
# 2. 아시아 국가들의 인구 변화 추세 (선 그래프)

asia_df = gapminder_df[gapminder_df['continent'] == 'Asia']

fig = px.line(asia_df,
              x= 'year',
              y= 'pop',
              color= 'country',
              title='아시아 국가들의 인구 변화 추세')

# 아시아 국가들만 필터링
asia_df = gapminder_df[gapminder_df['continent'] == 'Asia']

# 선 그래프 생성
fig = px.line(asia_df,
              x='year',
              y='pop',
              color='country',
              log_y=True,
              title='아시아 국가들의 인구 변화 추세',
              labels={'pop': '인구', 'year': '연도'})

fig.show()

In [27]:
# 3. 2007년 기준, 대륙별 평균 기대수명과 평균 1인당 GDP (막대 그래프)
# 또는 산점도로 표현

df_2007 = gapminder_df[gapminder_df['year'] == 2007]

# 대륙별 평균
continent_mean = df_2007.groupby('continent').agg({
    'lifeExp': 'mean',
    'gdpPercap': 'mean'
}).reset_index()

# 막대 - 합계 차트
# fig1 = px.bar(continent_mean,
#               x= 'continent',
#               y= ['lifeExp', 'gdpPercap'],
#               color= 'continent',
#               title= '2007년 대륙별 평균 기대수명과 1인당 GDP')

# 산점도 생성
fig2 = px.scatter(continent_mean,
                 x='gdpPercap',
                 y='lifeExp',
                 color='continent',
                 size='gdpPercap',
                 log_x= 'True',
                 title='2007년 대륙별 평균 기대수명과 1인당 GDP')

fig2.show()

In [32]:
# 4. 연도별 인구 Top 5 국가 변화 (애니메이션 막대 경주 - Plotly Express로는 직접 구현 어려움, graph_objects 필요)
# 이 부분은 고급 주제로, 여기서는 개념만 설명하고 간단한 정적 차트로 대체합니다.
# 실제 막대 경주 차트는 매년 순위가 바뀌는 것을 부드럽게 애니메이션 처리해야 합니다.

# 2007년 인구 Top 10 국가 (정적 막대 그래프)
top_country_2007 = gapminder_df[gapminder_df['year'] == 2007].nlargest(10, 'pop')

fig = px.bar(top_country_2007,
             x= 'country',
             y= 'pop',
             color= 'country',
             title= '2007년 인구 Top 10 국가',
             log_y= True)

fig.show()

### 🌟 도전 과제: 원하는 공공데이터로 시각화 도전 🚀 (발표자 3분 모십니다)

자신이 관심 있는 분야의 공공데이터(예: [공공데이터포털](https://www.data.go.kr/), [서울 열린 데이터 광장](https://data.seoul.go.kr/), [KOSIS 국가통계포털](https://kosis.kr/))를 찾아 다운로드하고, 다음과 같은 과정을 거쳐 자신만의 시각화 분석을 수행해보세요.

1.  **주제 선정 및 데이터 수집**: 어떤 질문에 답하고 싶은지, 어떤 현상을 탐색하고 싶은지 정의합니다.
2.  **데이터 전처리**: Pandas를 사용하여 데이터를 불러오고, 결측치 처리, 데이터 타입 변환, 필요한 파생 변수 생성 등의 작업을 수행합니다.
3.  **탐색적 데이터 분석(EDA) 및 시각화**:
      * 데이터의 기본적인 통계량 확인.
      * 다양한 Plotly 그래프(산점도, 막대, 선, 히스토그램, 파이, 지리 시각화 등)를 활용하여 데이터의 패턴, 관계, 분포, 이상치 등을 탐색합니다.
      * 대화형 기능(hover, 애니메이션, 슬라이더)을 적절히 활용하여 깊이 있는 분석을 시도합니다.
      * `make_subplots`나 `facet` 기능을 사용하여 여러 관점을 동시에 비교합니다.
4.  **결론 및 인사이트 도출**: 시각화 결과를 바탕으로 처음 설정했던 질문에 대한 답을 찾거나, 새로운 인사이트를 발견합니다.

이 과정은 데이터 분석가로서 매우 중요한 역량이며, Plotly는 이 과정을 시각적 표현으로 도와줍니다.

In [ ]:
# 예시:
# 1. 데이터셋 설명
#   - 데이터명: 서울시 지하철 호선별 역별 승하차 인원 정보
#   - 출처: 서울 열린 데이터 광장 (https://data.seoul.go.kr/)
#   - 내용: 특정 기간 동안 서울시 지하철 역들의 호선별, 시간대별 승하차 인원 수

# 2. 분석 목표
#   - 서울시에서 가장 붐비는 지하철 역과 호선은 어디인가?
#   - 출퇴근 시간대 주요 역들의 특징은 무엇인가?
#   - 주중과 주말의 이용 패턴 차이가 있는가?

# 3. 데이터 전처리 과정
#   (Pandas 코드 예시)
#   df_subway = pd.read_csv("서울시_지하철_승하차인원.csv", encoding='cp949')
#   # ... (결측치 처리, 타입 변환, 필요한 컬럼 생성 등)

# 4. 시각화 및 분석 결과
#   # 시각화 1: 호선별 총 승하차 인원 (막대 그래프)
#   # ... (px.bar 코드 및 fig.show())
#   # 분석: 2호선이 압도적으로 높은 승하차 인원을 보임...

#   # 시각화 2: 승하차 인원 상위 10개 역 (수평 막대 그래프)
#   # ... (px.bar 코드 및 fig.show())
#   # 분석: 강남, 잠실, 홍대입구 등이 상위권을 차지함...

#   # 시각화 3: 특정 역(예: 강남역)의 시간대별 평균 승하차 인원 (선 그래프 또는 막대 그래프)
#   # ... (px.line 또는 px.bar 코드 및 fig.show())
#   # 분석: 출근 시간(08-09시)과 퇴근 시간(18-19시)에 뚜렷한 피크를 보임...

# 5. 결론 및 제언
#   - 분석 결과, 2호선과 주요 환승역들의 이용객이 매우 많음을 확인.
#   - 특정 시간대 혼잡도 완화를 위한 정책적 고려가 필요할 수 있음.
#   - 향후 요일별, 공휴일 여부 등 추가 변수를 고려한 분석이 유용할 것.

In [10]:
!ls -al datasets/

total 88
drwxrwxrwx 1 root   root    512 Jun  2 06:38 .
drwsrwsr-x 1 jovyan users  4096 Jun  2 04:47 ..
-rwxrwxrwx 1 root   root   6849 Jun  2 06:33 criminal_occupation.csv
-rwxrwxrwx 1 root   root  73835 Jun  2 00:59 Pikachu.obj


In [11]:
import pandas as pd

### 데이터셋
- 데이터명: 경찰청 범죄 발생 장소별 통계 (2023)
- 출처 : 공공 데이터 포털 (https://www.data.go.kr/index.do)
- 내용: 범죄 장소별 발생 범죄 유형 통계

### 분석 목표
- 장소별 범죄 유형 발생 건수 (막대 그래프) -> 가장 범죄가 많이 발생하는 장소와 해당 장소에서 가장 많이 발생하는 범죄는?
- 범죄 유형별로 주로 발생하는 장소 (트리맵) -> 가장 많이 발생하는 범죄와 주로 발생하는 장소는?

In [71]:
# 경찰청 범죄발생 장소 데이터 불러오기
df_crime = pd.read_csv('datasets/경찰청_범죄발생장소_20221231.csv', encoding='cp949')
df_crime.head()

,범죄대분류,범죄중분류,아파트_연립다세대,단독주택,고속도로,노상,백화점,슈퍼마켓,편의점,대형할인매장,...,종교기관,산야,해상,부대,구금장소,공지,주차장,공중화장실,피씨방,기타
0,강력범죄,살인기수,114,60,0,28,0,0,1,0,...,1,3,3,0,0,0,5,0,0,17
1,강력범죄,살인미수등,123,85,0,105,0,1,0,1,...,1,3,0,0,0,0,5,2,1,33
2,강력범죄,강도,62,46,0,141,0,12,32,1,...,0,3,0,0,1,0,21,1,4,30
3,강력범죄,강간,1616,880,0,158,0,2,3,0,...,7,9,0,1,0,1,45,48,3,560
4,강력범죄,유사강간,240,147,2,57,1,0,0,0,...,2,5,0,0,2,0,7,12,1,111


In [72]:
# 데이터 전처리

# 1. 결측치 확인
print(df_crime.isnull().sum())

# 2. 데이터 타입 확인 
print(df_crime.dtypes)


범죄대분류        0
범죄중분류        0
아파트_연립다세대    0
단독주택         0
고속도로         0
노상           0
백화점          0
슈퍼마켓         0
편의점          0
대형할인매장       0
상점           0
시장_노점        0
숙박업소_목욕탕     0
유흥접객업소       0
사무실          0
공장           0
공사장_광산       0
창고           0
역_대합실        0
지하철          0
기타교통수단내      0
흥행장          0
유원지          0
학교           0
금융기관         0
의료기관         0
종교기관         0
산야           0
해상           0
부대           0
구금장소         0
공지           0
주차장          0
공중화장실        0
피씨방          0
기타           0
dtype: int64
범죄대분류        object
범죄중분류        object
아파트_연립다세대     int64
단독주택          int64
고속도로          int64
노상            int64
백화점           int64
슈퍼마켓          int64
편의점           int64
대형할인매장        int64
상점            int64
시장_노점         int64
숙박업소_목욕탕      int64
유흥접객업소        int64
사무실           int64
공장            int64
공사장_광산        int64
창고            int64
역_대합실         int64
지하철           int64
기타교통수단내       int64
흥행장           int64
유원지    

In [61]:
# 범죄 중분류 데이터 추출
print(df_crime['범죄중분류'].unique())

['살인기수' '살인미수등' '강도' '강간' '유사강간' '강제추행' '기타 강간_강제추행등' '방화' '절도범죄' '상해'
 '폭행' '체포_감금' '협박' '약취?유인' '폭력행위등' '공갈' '손괴' '직무유기' '직권남용' '증수뢰' '통화'
 '문서_인장' '유가증권인지' '사기' '횡령' '배임' '성풍속범죄' '도박범죄' '특별경제범죄' '마약범죄' '보건범죄'
 '환경범죄' '교통범죄' '노동범죄' '안보범죄' '선거범죄' '병역범죄' '기타범죄']


In [62]:
print(df_crime.columns)

Index(['범죄대분류', '범죄중분류', '아파트_연립다세대', '단독주택', '고속도로', '노상', '백화점', '슈퍼마켓',
       '편의점', '대형할인매장', '상점', '시장_노점', '숙박업소_목욕탕', '유흥접객업소', '사무실', '공장',
       '공사장_광산', '창고', '역_대합실', '지하철', '기타교통수단내', '흥행장', '유원지', '학교', '금융기관',
       '의료기관', '종교기관', '산야', '해상', '부대', '구금장소', '공지', '주차장', '공중화장실', '피씨방',
       '기타'],
      dtype='object')


### 1. 장소별 범죄 유형 발생 건수 (막대 그래프)

In [93]:
# 장소 컬럼 추출 (범죄대분류, 범죄중분류 제외)
location_columns = df_crime.columns[2:]

# 데이터 재구성
crime_by_loc = pd.melt(df_crime, 
                       id_vars=['범죄대분류', '범죄중분류'],
                       value_vars=location_columns,
                       var_name='장소',
                       value_name='발생건수')

# 막대 그래프 생성
fig = px.bar(crime_by_loc,
             x='장소',
             y='발생건수',
             color='범죄중분류',
             title='장소/범죄 유형별 발생 건수')

fig.update_layout(
    xaxis_title='장소',
    yaxis_title='발생 건수',
    xaxis_tickangle=45,
    width=1500,
    height=800
)

# 발생건수 기준으로 정렬
fig.update_xaxes(categoryorder='total descending')

fig.show()

### 2. 범죄 유형별로 주로 발생하는 장소 (트리맵)

In [90]:
# 범죄 유형별 장소 데이터 준비
crime_type_loc = crime_by_loc.groupby(['범죄중분류', '장소'])['발생건수'].sum().reset_index()

# 각 범죄 유형별로 상위 5개 장소 선택
top_locations = crime_type_loc.sort_values('발생건수', ascending=False).groupby('범죄중분류').head(5)

# 트리맵 생성
fig = px.treemap(top_locations,
                 path=['범죄중분류', '장소'],
                 values='발생건수',
                 title='범죄 유형별 주요 발생 장소',
                 color='발생건수',
                 color_continuous_scale='Viridis')

fig.update_layout(
    width=1500,
    height=800
)

fig.show()

### 3. 범죄 대분류별 발생 건수 (도넛 차트)

In [96]:
crime_by_major = crime_by_loc.groupby('범죄대분류')['발생건수'].sum().reset_index()

# 파이 차트로 범죄 대분류 비율 시각화
fig = px.pie(crime_by_major, 
             values='발생건수',
             names='범죄대분류',
             title='범죄 대분류별 발생 비율',
             hole=0.4)

fig.update_layout(
    width=800,
    height=600
)

fig.show()